### Build a Simple LLM Application with LCEL
This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting.

A high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [10]:
!pip install langchain

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
!pip install langchain_groq

In [5]:
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x72ac98513860>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x72ac98346840>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
!pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='The translation of "Hello How are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour:**  Hello\n* **comment:** how\n* **allez-vous:** are you (formal, polite)\n\n\nLet me know if you\'d like some other translations!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 21, 'total_tokens': 95, 'completion_time': 0.134545455, 'prompt_time': 0.002169355, 'queue_time': 0.031673932, 'total_time': 0.13671481}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6db10e47-de54-44ff-9493-12264c97bf56-0', usage_metadata={'input_tokens': 21, 'output_tokens': 74, 'total_tokens': 95})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'The translation of "Hello How are you?" in French is:\n\n**Bonjour, comment allez-vous ?** \n\nHere\'s a breakdown:\n\n* **Bonjour:**  Hello\n* **comment:** how\n* **allez-vous:** are you (formal, polite)\n\n\nLet me know if you\'d like some other translations!\n'

In [9]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you\'d like to know more ways to greet someone in French!\n'

In [11]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [12]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [13]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [14]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'